# Using Intake-ESM Catalog

This Jupyter notebook demonstrates how to work with intake-esm esm catalog for the NCAR Community Earth System Model (CESM)
Large Ensemble (LENS) data hosted on AWS S3 ([doi:10.26024/wt24-5j82](https://doi.org/10.26024/wt24-5j82)).

[Intake-esm Documentation](https://intake-esm.readthedocs.io/en/latest/notebooks/tutorial.html)


## Import packages

In [1]:
import intake
import pprint

## Inspect intake-esm catalog

In [2]:
# Open original collection description file
cat_url= 'https://ncar-cesm-lens.s3-us-west-2.amazonaws.com/catalogs/aws-cesm1-le.json'
col = intake.open_esm_datastore(cat_url)
col

,unique
component,5
frequency,6
experiment,4
variable,73
path,414
variable_long_name,70
dim_per_tstep,3
start,12
end,13


Show the first few lines of the catalog. There are as many lines as there are paths. The order is the same as that of the CSV catalog file listed in the JSON description file.

In [3]:
print("Catalog file:", col.esmcol_data['catalog_file'])
col.df.head(10)

Catalog file: https://ncar-cesm-lens.s3-us-west-2.amazonaws.com/catalogs/aws-cesm1-le.csv


,component,frequency,experiment,variable,path,variable_long_name,dim_per_tstep,start,end
0,atm,daily,20C,FLNS,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-FLNS....,net longwave flux at surface,2.0,1920-01-01 12:00:00,2005-12-31 12:00:00
1,atm,daily,20C,FLNSC,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-FLNSC...,clearsky net longwave flux at surface,2.0,1920-01-01 12:00:00,2005-12-31 12:00:00
2,atm,daily,20C,FLUT,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-FLUT....,upwelling longwave flux at top of model,2.0,1920-01-01 12:00:00,2005-12-31 12:00:00
3,atm,daily,20C,FSNS,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-FSNS....,net solar flux at surface,2.0,1920-01-01 12:00:00,2005-12-31 12:00:00
4,atm,daily,20C,FSNSC,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-FSNSC...,clearsky net solar flux at surface,2.0,1920-01-01 12:00:00,2005-12-31 12:00:00
5,atm,daily,20C,FSNTOA,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-FSNTO...,net solar flux at top of atmosphere,2.0,1920-01-01 12:00:00,2005-12-31 12:00:00
6,atm,daily,20C,ICEFRAC,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-ICEFR...,fraction of sfc area covered by sea-ice,2.0,1920-01-01 12:00:00,2005-12-31 12:00:00
7,atm,daily,20C,LHFLX,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-LHFLX...,surface latent heat flux,2.0,1920-01-01 12:00:00,2005-12-31 12:00:00
8,atm,daily,20C,PRECL,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-PRECL...,large-scale (stable) precipitation rate (liq +...,2.0,1920-01-01 12:00:00,2005-12-31 12:00:00
9,atm,daily,20C,PRECSC,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-PRECS...,convective snow rate (water equivalent),2.0,1920-01-01 12:00:00,2005-12-31 12:00:00


**Table:** *First few lines of the original Intake-ESM Catalog showing the model component, the temporal frequency, the experiment, the abbreviated variable name, and the AWS S3 path for each Zarr store.*

Look at unique values and their counts for given columns

In [4]:
uniques_orig = col.unique(columns=["component", "frequency", "experiment", "variable"])
pprint.pprint(uniques_orig, compact=True, indent=1, width=80)

{'component': {'count': 5, 'values': ['atm', 'ice_nh', 'ice_sh', 'lnd', 'ocn']},
 'experiment': {'count': 4, 'values': ['20C', 'CTRL', 'HIST', 'RCP85']},
 'frequency': {'count': 6,
               'values': ['daily', 'hourly6-1990-2005', 'hourly6-2026-2035',
                          'hourly6-2071-2080', 'monthly', 'static']},
 'variable': {'count': 73,
              'values': ['DIC', 'DOC', 'FLNS', 'FLNSC', 'FLUT', 'FSNO', 'FSNS',
                         'FSNSC', 'FSNTOA', 'FW', 'H2OSNO', 'HMXL', 'ICEFRAC',
                         'LHFLX', 'O2', 'PD', 'PRECC', 'PRECL', 'PRECSC',
                         'PRECSL', 'PRECT', 'PRECTMX', 'PS', 'PSL', 'Q', 'Q850',
                         'QFLUX', 'QREFHT', 'QRUNOFF', 'QSW_HBL', 'QSW_HTP',
                         'RAIN', 'RESID_S', 'SALT', 'SFWF', 'SFWF_WRST', 'SHF',
                         'SHFLX', 'SHF_QSW', 'SNOW', 'SOILLIQ',
                         'SOILWATER_10CM', 'SSH', 'SST', 'T', 'TAUX', 'TAUX2',
                         'TAUY'

## Finding Data

If you happen to know the meaning of the variable names, you can find what data are available for that variable. For example:

In [5]:
# Filter the catalog to find available data for one variable
col.search(variable='FLNS').df

,component,frequency,experiment,variable,path,variable_long_name,dim_per_tstep,start,end
0,atm,daily,20C,FLNS,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-FLNS....,net longwave flux at surface,2.0,1920-01-01 12:00:00,2005-12-31 12:00:00
1,atm,daily,CTRL,FLNS,s3://ncar-cesm-lens/atm/daily/cesmLE-CTRL-FLNS...,net longwave flux at surface,2.0,0402-01-01 12:00:00,2200-12-31 12:00:00
2,atm,daily,HIST,FLNS,s3://ncar-cesm-lens/atm/daily/cesmLE-HIST-FLNS...,net longwave flux at surface,1.0,1850-01-01 12:00:00,1919-12-31 12:00:00
3,atm,daily,RCP85,FLNS,s3://ncar-cesm-lens/atm/daily/cesmLE-RCP85-FLN...,net longwave flux at surface,2.0,2006-01-01 12:00:00,2100-12-31 12:00:00
4,atm,hourly6-1990-2005,20C,FLNS,s3://ncar-cesm-lens/atm/hourly6-1990-2005/cesm...,net longwave flux at surface,2.0,1990-01-01 00:00:00,2006-01-01 00:00:00
5,atm,hourly6-2026-2035,RCP85,FLNS,s3://ncar-cesm-lens/atm/hourly6-2026-2035/cesm...,net longwave flux at surface,2.0,2026-01-01 00:00:00,2036-01-01 00:00:00
6,atm,hourly6-2071-2080,RCP85,FLNS,s3://ncar-cesm-lens/atm/hourly6-2071-2080/cesm...,net longwave flux at surface,2.0,2071-01-01 00:00:00,2081-01-01 00:00:00
7,atm,monthly,20C,FLNS,s3://ncar-cesm-lens/atm/monthly/cesmLE-20C-FLN...,net longwave flux at surface,2.0,1920-01-16 12:00:00,2005-12-16 12:00:00
8,atm,monthly,CTRL,FLNS,s3://ncar-cesm-lens/atm/monthly/cesmLE-CTRL-FL...,net longwave flux at surface,2.0,0400-01-16 12:00:00,2200-12-16 12:00:00
9,atm,monthly,HIST,FLNS,s3://ncar-cesm-lens/atm/monthly/cesmLE-HIST-FL...,net longwave flux at surface,1.0,1850-01-16 12:00:00,1919-12-16 12:00:00


**Table:** *All available Zarr stores for the "FLNS" data.*

In [6]:
# Narrow the filter to specific frequency and expiriment
col.search(variable='FLNS', frequency='daily', experiment='RCP85').df

,component,frequency,experiment,variable,path,variable_long_name,dim_per_tstep,start,end
0,atm,daily,RCP85,FLNS,s3://ncar-cesm-lens/atm/daily/cesmLE-RCP85-FLN...,net longwave flux at surface,2.0,2006-01-01 12:00:00,2100-12-31 12:00:00


**Table:** *The single Zarr store for daily "FLNS" data from "RCP85" experiment.*

## The Problem

Do all potential users know that "FLNS" is a CESM-specific abbreviation for "Net longwave flux at surface"? How would a novice user find out, other than by finding separate documentation, or by opening a Zarr store in the hopes that the long name might be recorded there? How do we address the fact that every climate model code seems to have a different, non-standard name for all the variables, thus making multi-source research needlessly difficult?

## Solution: use columns with enhanced information

By adding additional columns to the Intake-ESM catalog, we should be able to improve semantic interoperability and provide potentially useful information to the users.

### Long names

Instead of searching by variable short names, we can use the `variable_long_name`. 

**ISSUE:** *The long names are **not** CF Standard Names, but rather are those documented at http://www.cgd.ucar.edu/ccr/strandwg/CESM-CAM5-BGC_LENS_fields.html. For interoperability, the `variable_long_name` column should be supplemented by a `cf_name` column and possibly an attribute column to disambiguate if needed.*

In [7]:
col.df.head(10)

,component,frequency,experiment,variable,path,variable_long_name,dim_per_tstep,start,end
0,atm,daily,20C,FLNS,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-FLNS....,net longwave flux at surface,2.0,1920-01-01 12:00:00,2005-12-31 12:00:00
1,atm,daily,20C,FLNSC,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-FLNSC...,clearsky net longwave flux at surface,2.0,1920-01-01 12:00:00,2005-12-31 12:00:00
2,atm,daily,20C,FLUT,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-FLUT....,upwelling longwave flux at top of model,2.0,1920-01-01 12:00:00,2005-12-31 12:00:00
3,atm,daily,20C,FSNS,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-FSNS....,net solar flux at surface,2.0,1920-01-01 12:00:00,2005-12-31 12:00:00
4,atm,daily,20C,FSNSC,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-FSNSC...,clearsky net solar flux at surface,2.0,1920-01-01 12:00:00,2005-12-31 12:00:00
5,atm,daily,20C,FSNTOA,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-FSNTO...,net solar flux at top of atmosphere,2.0,1920-01-01 12:00:00,2005-12-31 12:00:00
6,atm,daily,20C,ICEFRAC,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-ICEFR...,fraction of sfc area covered by sea-ice,2.0,1920-01-01 12:00:00,2005-12-31 12:00:00
7,atm,daily,20C,LHFLX,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-LHFLX...,surface latent heat flux,2.0,1920-01-01 12:00:00,2005-12-31 12:00:00
8,atm,daily,20C,PRECL,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-PRECL...,large-scale (stable) precipitation rate (liq +...,2.0,1920-01-01 12:00:00,2005-12-31 12:00:00
9,atm,daily,20C,PRECSC,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-PRECS...,convective snow rate (water equivalent),2.0,1920-01-01 12:00:00,2005-12-31 12:00:00


In [8]:
# List all available variables by Long Name, sorted alphabetically
uniques = col.unique(columns=['variable_long_name'])
namelist = sorted(uniques['variable_long_name']['values'])
[x for x in namelist]

['atmospheric rain',
 'atmospheric snow',
 'clearsky net longwave flux at surface',
 'clearsky net solar flux at surface',
 'convective precipitation rate (liq + ice)',
 'convective snow rate (water equivalent)',
 'dissolved inorganic carbon',
 'dissolved organic carbon',
 'dissolved oxygen',
 'flux of heat in grid-x direction',
 'flux of heat in grid-y direction',
 'fraction of ground covered by snow',
 'fraction of sfc area covered by sea-ice',
 'free-surface residual flux (s)',
 'freshwater flux',
 'geopotential height (above sea level)',
 'geopotential z at 500 mbar pressure surface',
 'grid cell mean ice thickness',
 'heat flux across top face',
 'horizontal total wind speed average at the surface',
 'ice area  (aggregate)',
 'internal ocean heat flux due to ice formation; heat of fusion > 0 or ice-melting potential < 0',
 'large-scale (stable) precipitation rate (liq + ice)',
 'large-scale (stable) snow rate (water equivalent)',
 'lowest model level zonal wind',
 'maximum (convec

In [9]:
# Show all available data for a specific variable based on long name
varname = 'salinity'
col.search(variable_long_name=varname).df

,component,frequency,experiment,variable,path,variable_long_name,dim_per_tstep,start,end
0,ocn,monthly,20C,SALT,s3://ncar-cesm-lens/ocn/monthly/cesmLE-20C-SAL...,salinity,3.0,1920-01-16 12:00:00,2005-12-16 12:00:00
1,ocn,monthly,CTRL,SALT,s3://ncar-cesm-lens/ocn/monthly/cesmLE-CTRL-SA...,salinity,3.0,0400-01-16 12:00:00,2200-12-16 12:00:00
2,ocn,monthly,HIST,SALT,s3://ncar-cesm-lens/ocn/monthly/cesmLE-HIST-SA...,salinity,2.0,1850-01-16 12:00:00,1919-12-16 12:00:00
3,ocn,monthly,RCP85,SALT,s3://ncar-cesm-lens/ocn/monthly/cesmLE-RCP85-S...,salinity,3.0,2006-01-16 12:00:00,2100-12-16 12:00:00


**Table:** *All available data in this catalog for selected variable*

### Substring matches

With use of wildcards and/or regular expressions, we can find all variables with a particular substring in the long name:

In [10]:
# Find all entries with `wind` in their variable_long_name
col.search(variable_long_name='wind*').df

,component,frequency,experiment,variable,path,variable_long_name,dim_per_tstep,start,end
0,atm,daily,20C,UBOT,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-UBOT....,lowest model level zonal wind,2.0,1920-01-01 12:00:00,2005-12-31 12:00:00
1,atm,daily,20C,WSPDSRFAV,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-WSPDS...,horizontal total wind speed average at the sur...,2.0,1920-01-01 12:00:00,2005-12-31 12:00:00
2,atm,daily,HIST,UBOT,s3://ncar-cesm-lens/atm/daily/cesmLE-HIST-UBOT...,lowest model level zonal wind,1.0,1850-01-01 12:00:00,1919-12-31 12:00:00
3,atm,daily,HIST,WSPDSRFAV,s3://ncar-cesm-lens/atm/daily/cesmLE-HIST-WSPD...,horizontal total wind speed average at the sur...,1.0,1850-01-01 12:00:00,1919-12-31 12:00:00
4,atm,hourly6-1990-2005,20C,U,s3://ncar-cesm-lens/atm/hourly6-1990-2005/cesm...,zonal wind,3.0,1990-01-01 00:00:00,2006-01-01 00:00:00
5,atm,hourly6-2026-2035,RCP85,U,s3://ncar-cesm-lens/atm/hourly6-2026-2035/cesm...,zonal wind,3.0,2026-01-01 00:00:00,2036-01-01 00:00:00
6,atm,hourly6-2071-2080,RCP85,U,s3://ncar-cesm-lens/atm/hourly6-2071-2080/cesm...,zonal wind,3.0,2071-01-01 00:00:00,2081-01-01 00:00:00
7,atm,monthly,20C,U,s3://ncar-cesm-lens/atm/monthly/cesmLE-20C-U.zarr,zonal wind,3.0,1920-01-16 12:00:00,2005-12-16 12:00:00
8,atm,monthly,CTRL,U,s3://ncar-cesm-lens/atm/monthly/cesmLE-CTRL-U....,zonal wind,3.0,0400-01-16 12:00:00,2200-12-16 12:00:00
9,atm,monthly,HIST,U,s3://ncar-cesm-lens/atm/monthly/cesmLE-HIST-U....,zonal wind,2.0,1850-01-16 12:00:00,1919-12-16 12:00:00


In [11]:
# Find all entries whose variable long name starts with `wind`
col.search(variable_long_name='^wind').df

,component,frequency,experiment,variable,path,variable_long_name,dim_per_tstep,start,end
0,ocn,monthly,20C,TAUX,s3://ncar-cesm-lens/ocn/monthly/cesmLE-20C-TAU...,windstress in grid-x direction,2.0,1920-01-16 12:00:00,2005-12-16 12:00:00
1,ocn,monthly,20C,TAUX2,s3://ncar-cesm-lens/ocn/monthly/cesmLE-20C-TAU...,windstress**2 in grid-x direction,2.0,1920-01-16 12:00:00,2005-12-16 12:00:00
2,ocn,monthly,20C,TAUY,s3://ncar-cesm-lens/ocn/monthly/cesmLE-20C-TAU...,windstress in grid-y direction,2.0,1920-01-16 12:00:00,2005-12-16 12:00:00
3,ocn,monthly,20C,TAUY2,s3://ncar-cesm-lens/ocn/monthly/cesmLE-20C-TAU...,windstress**2 in grid-y direction,2.0,1920-01-16 12:00:00,2005-12-16 12:00:00
4,ocn,monthly,CTRL,TAUX,s3://ncar-cesm-lens/ocn/monthly/cesmLE-CTRL-TA...,windstress in grid-x direction,2.0,0400-01-16 12:00:00,2200-12-16 12:00:00
5,ocn,monthly,CTRL,TAUX2,s3://ncar-cesm-lens/ocn/monthly/cesmLE-CTRL-TA...,windstress**2 in grid-x direction,2.0,0400-01-16 12:00:00,2200-12-16 12:00:00
6,ocn,monthly,CTRL,TAUY,s3://ncar-cesm-lens/ocn/monthly/cesmLE-CTRL-TA...,windstress in grid-y direction,2.0,0400-01-16 12:00:00,2200-12-16 12:00:00
7,ocn,monthly,CTRL,TAUY2,s3://ncar-cesm-lens/ocn/monthly/cesmLE-CTRL-TA...,windstress**2 in grid-y direction,2.0,0400-01-16 12:00:00,2200-12-16 12:00:00
8,ocn,monthly,HIST,TAUX,s3://ncar-cesm-lens/ocn/monthly/cesmLE-HIST-TA...,windstress in grid-x direction,1.0,1850-01-16 12:00:00,1919-12-16 12:00:00
9,ocn,monthly,HIST,TAUX2,s3://ncar-cesm-lens/ocn/monthly/cesmLE-HIST-TA...,windstress**2 in grid-x direction,1.0,1850-01-16 12:00:00,1919-12-16 12:00:00


**Table(s):** *Information about all matching datasets*

## Loading data into xarray datasets

Once we are satisfied with the results of our searches, we can use the `to_dataset_dict()` method to load the data into xarray datasets.

In [12]:
dsets = col.search(variable_long_name='temp*', frequency='monthly', experiment='20C', component='ocn').to_dataset_dict(zarr_kwargs={'consolidated': True})
dsets


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.frequency'


/Users/abanihi/opt/miniconda3/envs/intake-esm-dev/lib/python3.8/site-packages/dask/array/core.py:4027: PerformanceWarning: Increasing number of chunks by factor of 160
  result = blockwise(


{'ocn.20C.monthly': <xarray.Dataset>
 Dimensions:     (d2: 2, member_id: 40, nlat: 384, nlon: 320, time: 1032, z_t: 60)
 Coordinates:
   * z_t         (z_t) float64 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
   * member_id   (member_id) int64 1 2 3 4 5 6 7 8 ... 34 35 101 102 103 104 105
   * time        (time) object 1920-01-16 12:00:00 ... 2005-12-16 12:00:00
     time_bound  (time, d2) object dask.array<chunksize=(1032, 2), meta=np.ndarray>
 Dimensions without coordinates: d2, nlat, nlon
 Data variables:
     SST         (member_id, time, z_t, nlat, nlon) float32 dask.array<chunksize=(1, 300, 1, 384, 320), meta=np.ndarray>
     TEMP        (member_id, time, z_t, nlat, nlon) float32 dask.array<chunksize=(1, 6, 60, 384, 320), meta=np.ndarray>
 Attributes:
     Conventions:               CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netc...
     tavg_sum:                  2592000.0
     revision:                  $Id: tavg.F90 41939 2012-11-14 16:37:23Z mlevy...
     tavg_sum_qflux:    

### Grid variables

Grid variables, including the latitudes and longitudes of tracer points, do not have variable names or long names. So, to find them we need to use the `frequency='static` query:

In [13]:
col.search(frequency='static').df

,component,frequency,experiment,variable,path,variable_long_name,dim_per_tstep,start,end
0,atm,static,CTRL,NaN,s3://ncar-cesm-lens/atm/static/grid.zarr,NaN,NaN,NaN,NaN
1,atm,static,HIST,NaN,s3://ncar-cesm-lens/atm/static/grid.zarr,NaN,NaN,NaN,NaN
2,atm,static,RCP85,NaN,s3://ncar-cesm-lens/atm/static/grid.zarr,NaN,NaN,NaN,NaN
3,atm,static,20C,NaN,s3://ncar-cesm-lens/atm/static/grid.zarr,NaN,NaN,NaN,NaN
4,ice_nh,static,CTRL,NaN,s3://ncar-cesm-lens/ice_nh/static/grid.zarr,NaN,NaN,NaN,NaN
5,ice_nh,static,HIST,NaN,s3://ncar-cesm-lens/ice_nh/static/grid.zarr,NaN,NaN,NaN,NaN
6,ice_nh,static,RCP85,NaN,s3://ncar-cesm-lens/ice_nh/static/grid.zarr,NaN,NaN,NaN,NaN
7,ice_nh,static,20C,NaN,s3://ncar-cesm-lens/ice_nh/static/grid.zarr,NaN,NaN,NaN,NaN
8,ice_sh,static,CTRL,NaN,s3://ncar-cesm-lens/ice_sh/static/grid.zarr,NaN,NaN,NaN,NaN
9,ice_sh,static,HIST,NaN,s3://ncar-cesm-lens/ice_sh/static/grid.zarr,NaN,NaN,NaN,NaN


In [14]:
# To load grid variables for a specific component and experiment
_, grid = col.search(frequency="static", component="ocn", experiment="20C").to_dataset_dict(aggregate=False, zarr_kwargs={'consolidated': True}).popitem()
grid


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.frequency.experiment.variable.path.variable_long_name.dim_per_tstep.start.end'


<xarray.Dataset>
Dimensions:               (lat_aux_grid: 395, moc_comp: 3, moc_z: 61, nlat: 384, nlon: 320, transport_comp: 5, transport_reg: 2, z_t: 60, z_t_150m: 15, z_w: 60, z_w_bot: 60, z_w_top: 60)
Coordinates:
    ANGLE                 (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    ANGLET                (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    DXT                   (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    DXU                   (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    DYT                   (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    DYU                   (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    HT                    (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    HTE                   (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    HTN                   (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    HU                    (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    HUS                   (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    HUW                   (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    KMT                   (nlat, nlon) float64 dask.array<chunksize=(192, 320), meta=np.ndarray>
    KMU                   (nlat, nlon) float64 dask.array<chunksize=(192, 320), meta=np.ndarray>
    REGION_MASK           (nlat, nlon) float64 dask.array<chunksize=(192, 320), meta=np.ndarray>
    T0_Kelvin             float64 ...
    TAREA                 (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    TLAT                  (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    TLONG                 (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    UAREA                 (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    ULAT                  (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    ULONG                 (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    cp_air                float64 ...
    cp_sw                 float64 ...
    days_in_norm_year     timedelta64[ns] ...
    dz                    (z_t) float32 dask.array<chunksize=(60,), meta=np.ndarray>
    dzw                   (z_w) float32 dask.array<chunksize=(60,), meta=np.ndarray>
    fwflux_factor         float64 ...
    grav                  float64 ...
    heat_to_PW            float64 ...
    hflux_factor          float64 ...
  * lat_aux_grid          (lat_aux_grid) float32 -79.48815 -78.952896 ... 90.0
    latent_heat_fusion    float64 ...
    latent_heat_vapor     float64 ...
    mass_to_Sv            float64 ...
    moc_components        (moc_comp) |S256 dask.array<chunksize=(3,), meta=np.ndarray>
  * moc_z                 (moc_z) float32 0.0 1000.0 ... 525000.94 549999.06
    momentum_factor       float64 ...
    nsurface_t            float64 ...
    nsurface_u            float64 ...
    ocn_ref_salinity      float64 ...
    omega                 float64 ...
    ppt_to_salt           float64 ...
    radius                float64 ...
    rho_air               float64 ...
    rho_fw                float64 ...
    rho_sw                float64 ...
    salinity_factor       float64 ...
    salt_to_Svppt         float64 ...
    salt_to_mmday         float64 ...
    salt_to_ppt           float64 ...
    sea_ice_salinity      float64 ...
    sflux_factor          float64 ...
    sound                 float64 ...
    stefan_boltzmann      float64 ...
    transport_components  (transport_comp) |S256 dask.array<chunksize=(5,), meta=np.ndarray>
    transport_regions     (transport_reg) |S256 dask.array<chunksize=(2,), meta=np.ndarray>
    vonkar                float64 ...
